In [4]:
# main_detection.ipynb

# --- セットアップ ---
import sys
import cv2
import numpy as np
from pathlib import Path
import os

# src ディレクトリをモジュール検索パスに追加
# これにより、srcディレクトリ配下のモジュールをimportできる
ROOT_DIR = Path(os.getcwd()).parent
SRC_DIR = ROOT_DIR / "src"
sys.path.append(str(SRC_DIR))

# 自作モジュールのインポート
from camera_handler import CameraHandler              # カメラ入力処理
from model_handler import ModelHandler                # YOLOモデルの読み込みと推論処理
from detection_utils import DetectionVisualizer, DetectionFilter  # 描画・フィルタ処理
from config import DEFAULT_MODEL                      # デフォルトモデル名の定義

# --- コンポーネント初期化 ---
camera = CameraHandler()                              # カメラの初期化
model = ModelHandler(model_name=DEFAULT_MODEL)        # YOLOモデルの初期化
visualizer = DetectionVisualizer(language="en")       # 描画用（日本語表示したい場合は"jp"）
filterer = DetectionFilter()                          # 検出結果フィルタリング

# カメラとモデルの初期化確認
assert camera.initialize_camera(), "カメラ初期化失敗"
assert model.load_model(), "モデル読み込み失敗"

# --- メインループ（リアルタイム検出処理） ---
print("ESCキーで終了します")
printed_shape = False  # 画像サイズの一度だけ出力するためのフラグ

try:
    while True:
        ret, frame = camera.read_frame()              # カメラからフレームを取得
        if not ret:
            continue

        # 最初の1回だけ画像サイズを出力
        if not printed_shape:
            print(f"取得フレームサイズ: {frame.shape}")
            printed_shape = True

        # モデルの入力サイズにリサイズ（YOLOv8nの場合 1280×720）
        frame_for_inference = cv2.resize(frame, (1280, 720))

        # 推論実行（結果がNoneの場合はスキップ）
        result = model.predict(frame_for_inference)
        if result is None:
            continue

        # 検出結果に対してフィルタ処理（信頼度・重複除去など）
        detections = filterer.apply_all_filters(result["detections"])

        # 検出結果をフレームに描画
        vis_frame = visualizer.draw_detections(frame_for_inference, detections)

        # FPSや検出数などの統計情報をフレームに重ねる
        vis_frame = visualizer.add_info_overlay(
            vis_frame, 
            fps=camera.get_fps_stats()["actual_fps"], 
            detection_count=len(detections),
            model_name=model.model_name,
            processing_time=result["inference_time"] * 1000  # 推論時間 [ms]
        )

        # 結果をウィンドウに表示
        cv2.namedWindow("YOLO Real-time Detection", cv2.WINDOW_NORMAL)
        cv2.imshow("YOLO Real-time Detection", vis_frame)

        # ESCキーでループを終了（キーコード27）
        if cv2.waitKey(1) & 0xFF == 27: 
            break

finally:
    # 終了処理：カメラとウィンドウを解放
    camera.release()
    cv2.destroyAllWindows()


GPU使用: NVIDIA GeForce RTX 4070 Ti
カメラ初期化成功:
  解像度: 640x480 (要求: 1920x1080)
  FPS: 30 (要求: 30)
モデル読み込み中: yolov8n.pt
モデル読み込み完了 - デバイス: cuda, クラス数: 80
ESCキーで終了します
取得フレームサイズ: (480, 640, 3)
カメラリソースを解放しました
